Final results of `Linked Clusters`. Such clusters are created by `linking.py` and are representing signatures which could be linked back to authority records on CDS (author profiles), using Beard for the clustering. The presented results are based on the first approach, which implements a simple algorithm to process `Predicted Clusters` (output of the clustering). Having this results, the algorithm can be further improved.

In [35]:
import pandas as pd

from json import load

In [36]:
# Load signatures used for lookup the full signature for a given signature id
signatures = load(open("/data01/beard/clustering/signatures.json"))
signatures = {s["signature_id"]: s for s in signatures}
print len(signatures)

8958311


In [47]:
# Load linked clusters created by using different linking modes
linked_clusters_pessimistic = load(open("/data01/beard/linking/linked_clusters_pessimistic.json"))
linked_clusters_careful = load(open("/data01/beard/linking/linked_clusters_careful.json"))
linked_clusters_fair = load(open("/data01/beard/linking/linked_clusters_fair.json"))
linked_clusters_adventurous = load(open("/data01/beard/linking/linked_clusters_adventurous.json"))

In [48]:
def analyze_clusters(linked_clusters):
    number_of_signatures = 0
    number_of_signatures_having_authority_id = 0
    number_of_documents = set()

    for c in linked_clusters.itervalues():
        number_of_signatures += len(c)
        for id_ in c:
            if (signatures[id_].get("author_cds_id") or 
                signatures[id_].get("author_cern_id") or
                signatures[id_].get("author_inspire_id")):
                number_of_signatures_having_authority_id += 1
            number_of_documents.add(signatures[id_]["publication_id"])

    return (number_of_signatures,
            number_of_signatures_having_authority_id,
            number_of_documents)

In [10]:
# Get data for `pessimistic` mode
p_s_total, p_s_ids, p_docs = analyze_clusters(linked_clusters_pessimistic)
p_s_no_ids = p_s_total - p_s_ids
p_c = len(linked_clusters_pessimistic)

In [11]:
# Get data for `careful` mode
c_s_total, c_s_ids, c_docs = analyze_clusters(linked_clusters_careful)
c_s_no_ids = c_s_total - c_s_ids
c_c = len(linked_clusters_careful)

In [49]:
# Get data for `fair` mode
f_s_total, f_s_ids, f_docs = analyze_clusters(linked_clusters_fair)
f_s_no_ids = f_s_total - f_s_ids
f_c = len(linked_clusters_fair)

In [13]:
# Get data for `adventurous` mode
a_s_total, a_s_ids, a_docs = analyze_clusters(linked_clusters_adventurous)
a_s_no_ids = a_s_total - a_s_ids
a_c = len(linked_clusters_adventurous)

In [14]:
# Print results
data = [('pessimistic', [p_s_no_ids, p_s_ids, p_s_total, p_c, len(p_docs)]),
        ('careful', [c_s_no_ids, c_s_ids, c_s_total, c_c, len(c_docs)]),
        ('fair', [f_s_no_ids, f_s_ids, f_s_total, f_c, len(f_docs)]),
        ('adventurous', [a_s_no_ids, a_s_ids, a_s_total, a_c, len(a_docs)])]

columns = ['s without authority id',  # Number of signatures having no authority id
           's with authority id',  # Number of signatures having an authority id
           'total',  # Total number of signatures
           'clusters',  # Number of linked clusters
           'documents',  # Number of documents linked signatures belong to
          ]

df = pd.DataFrame.from_items(data, orient='index', columns=columns)
df

,s without authority id,s with authority id,total,clusters,documents
pessimistic,0,2327830,2327830,9469,5760
careful,314921,2350719,2665640,9469,58855
fair,425868,2377584,2803452,9469,68429
adventurous,1123729,2388806,3512535,9469,194201


The shown results are based on our first implementation of the `linking` algorithm, which is mainly based on authority id matching. However, considering the mode `fair`, we could link more than 400'000 signatures without ids back to an author.

Let us analyze the results we would have without using any `clustering` algorithms to disambiguate our set of signatures:

In [37]:
# Load look-up: for a given authority id, get the authority record id pointing to the profile on CDS
lookup = load(open("/data01/beard/linking/profile_lookup.json"))
len(lookup)

91036

Example usage:

In [31]:
lookup['AUTHOR|(SzGeCERN)389900']

u'2108556'

In [51]:
linked_signatures = {}
publications = set()

def add_signature(rec_id, signature):
    try:
        linked_signatures[rec_id].append(signature['signature_id'])
    except KeyError:
        linked_signatures[rec_id] = [signature['signature_id']]
    publications.add(signature['publication_id'])

for signature in signatures.itervalues():
    try:
        rec_id = lookup['AUTHOR|(CDS){}'.format(signature['author_cds_id'])]
        add_signature(rec_id, signature)
        continue
    except KeyError:
        pass
    
    try:
        rec_id = lookup['AUTHOR|(SzGeCERN){}'.format(signature['author_cern_id'])]
        add_signature(rec_id, signature)
        continue
    except KeyError:
        pass
    
    try:
        rec_id = lookup['AUTHOR|(INSPIRE)INSPIRE-{}'.format(signature['author_inspire_id'])]
        add_signature(rec_id, signature)
        continue
    except KeyError:
        pass

Lets compare the results of linking signatures by authority ids only (without using disambiguation) with the results presented above (using disambiguation and applying linking mode `fair`):

In [64]:
# Print results
data = [('Without disambiguation', [sum([len(s) for s in linked_signatures.itervalues()]),
                                    len(linked_signatures),
                                    len(publications)]),
        ('With disambiguation (`fair`)', [f_s_total, f_c, len(f_docs)]),
       ]
columns = ['#Signatures',  # Number of signatures having authority ids
           '#Clusters',  # Number of clusters containing signature with authority id
           '#Documents', # Number of documents the signatures are contributors of
          ]
df = pd.DataFrame.from_items(data, orient='index', columns=columns)
df

,#Signatures,#Clusters,#Documents
Without disambiguation,2503432,9594,6012
With disambiguation (`fair`),2803452,9469,68429


The table shows that a small number of clusters has not been matched using our implemented (simple) algorithm. That being said, that `linking.py` has to be improved in future work. But it also shows, that more signatures having no authority id could be linked (see example clusters below), as well as the number of affected documents could increased significantly.

In [63]:
print len(linked_signatures['2108556'])
linked_signatures['2108556']

5


[u'500251_Ellis, Jonathan Richard_1751065',
 u'1262925_Ellis, John_4155994',
 u'1233863_Ellis, J R_4075997',
 u'2132991_Ellis, John R_8731810',
 u'1107496_Ellis, John R._3683610']

In [62]:
print len(linked_clusters_fair['2108556'])
linked_clusters_fair['2108556']

1051


[u'563568_Ellis, Jonathan Richard_1927045',
 u'246997_Ellis, Jonathan Richard_632292',
 u'228045_Ellis, Jonathan Richard_531673',
 u'134194_Ellis, Jonathan Richard_88358',
 u'2141265_Ellis, John_8834019',
 u'147292_Ellis, Jonathan Richard_140970',
 u'159669_Ellis, Jonathan Richard_190469',
 u'974133_Ellis, Jonathan Richard_3134333',
 u'1038913_Ellis, Jonathan Richard_3396899',
 u'735006_Ellis, Jonathan Richard_2312365',
 u'143433_Ellis, Jonathan Richard_126513',
 u'236713_Ellis, Jonathan Richard_564431',
 u'567709_Ellis, Jonathan Richard_1934744',
 u'301580_Ellis, Jonathan Richard_956511',
 u'101241_Ellis, Jonathan Richard_14656',
 u'489269_Ellis, Jonathan Richard_1719892',
 u'876877_Ellis, Jonathan Richard_2811075',
 u'173736_Ellis, Jonathan Richard_249048',
 u'236521_Ellis, Jonathan Richard_562353',
 u'427633_Ellis, Jonathan Richard_1510871',
 u'241523_Ellis, Jonathan Richard_595744',
 u'324821_Ellis, Jonathan Richard_1076943',
 u'968167_Ellis, John_3116899',
 u'166918_Ellis, Jonatha